# Netflix - Phone Partnership Subscriber Retention Metrics

```SQL
DROP TABLE IF EXISTS dim_telecom_partners;

DROP TABLE IF EXISTS fct_bundle_subscriptions;

CREATE TABLE dim_telecom_partners (
    partner_id INTEGER,
    partner_name VARCHAR
);

CREATE TABLE fct_bundle_subscriptions (
    subscriber_id INTEGER,
    partner_id INTEGER,
    bundle_id INTEGER,
    conversion_date DATE,
    retention_days INTEGER,
    engagement_score DECIMAL
);

INSERT INTO dim_telecom_partners (partner_id, partner_name)
VALUES
    (1, 'Verizon'),
    (2, 'AT&T'),
    (3, 'T-Mobile'),
    (4, 'Sprint');

INSERT INTO fct_bundle_subscriptions (
    subscriber_id,
    partner_id,
    bundle_id,
    conversion_date,
    retention_days,
    engagement_score
)
VALUES
    (1, 1, 1, '2024-01-05', 30, 7.5),
    (2, 2, 1, '2024-01-15', 25, 8.0),
    (3, 1, 2, '2024-01-20', 40, 7.8),
    (4, 3, 1, '2024-01-25', 20, 8.5),
    (5, 1, 3, '2024-01-30', 35, 7.0),
    (6, 2, 2, '2024-10-09', 60, 9.0),
    (7, 1, 1, '2024-10-10', 45, 8.7),
    (8, 3, 3, '2024-10-12', 75, 7.9),
    (9, 2, 4, '2024-10-13', 80, 8.2),
    (10, 1, 2, '2024-10-14', 50, 8.1),
    (11, 3, 1, '2024-11-03', 30, 7.5),
    (12, 1, 2, '2024-11-10', 45, 8.3),
    (13, 3, 2, '2024-11-15', 55, 7.2),
    (14, 2, 3, '2024-11-20', 40, 8.0),
    (15, 3, 4, '2024-11-25', 65, 8.6),
    (15, 3, 4, '2024-11-26', 65, 1.2),
    (16, 1, 4, '2024-12-05', 30, 7.8);

SELECT * FROM dim_telecom_partners;
SELECT * FROM fct_bundle_subscriptions;
```

In [1]:
import pandas as pd
import numpy as np

In [4]:
df_telecom = pd.read_csv('Data/004/dim_telecom_partners.csv')
df_bundle = pd.read_csv('Data/004/fct_bundle_subscriptions.csv', parse_dates=['conversion_date'])

df_telecom.head()

,partner_id,partner_name
0,1,Verizon
1,2,AT&T
2,3,T-Mobile
3,4,Sprint


In [5]:
df_bundle.head()

,subscriber_id,partner_id,bundle_id,conversion_date,retention_days,engagement_score
0,1,1,1,2024-01-05,30,7.5
1,2,2,1,2024-01-15,25,8.0
2,3,1,2,2024-01-20,40,7.8
3,4,3,1,2024-01-25,20,8.5
4,5,1,3,2024-01-30,35,7.0


# Pregunta 1

#### Para los suscriptores que se convirtieron (se unieron/suscribieron) en enero de 2024, ¿cuál es el nombre del socio de Telecomunicaciones (Telecom partner) que lideró la adquisición de la mayor cantidad de nuevos suscriptores?

In [10]:
df_merge = df_telecom.merge(df_bundle, on='partner_id', how='inner')

df_enero = df_merge[
    (df_merge['conversion_date'].between('2024-01-01','2024-01-31'))
].copy()

respuesta = df_enero.groupby('partner_name')['subscriber_id'].count().reset_index(name='total_subscribers')

respuesta1 = respuesta.sort_values(by='total_subscribers', ascending=False).head(1)

respuesta1

,partner_name,total_subscribers
2,Verizon,3


```SQL
SELECT
    d.partner_name,
    COUNT(f.subscriber_id) AS total_subscribers
FROM dim_telecom_partners d
INNER JOIN fct_bundle_subscriptions f ON d.partner_id = f.partner_id
WHERE f.conversion_date BETWEEN '2024-01-01' AND '2024-01-31'
GROUP BY d.partner_name
ORDER BY total_subscribers DESC
LIMIT 1;
```

# Pregunta 2

### Para cada socio de telecomunicaciones, ¿cuál es el mayor número de días que un suscriptor permaneció activo después de la conversión y a qué paquete(s) se suscribió? Para este análisis, considera únicamente las conversiones entre el 8 y el 14 de octubre de 2024. Si hay varios paquetes que resultaron en la misma retención máxima, devuelve todos esos paquetes


In [19]:
# Cambiamos el '2024-10-01' por '2024-10-08'
df_oct = df_merge[df_merge['conversion_date'].between('2024-10-08','2024-10-14')].copy()

# El resto está perfecto
max_days = df_oct.groupby('partner_name')['retention_days'].transform('max')
resultado = df_oct[df_oct['retention_days'] == max_days][['partner_name', 'retention_days', 'bundle_id']]

resultado

,partner_name,retention_days,bundle_id
4,Verizon,50,2
9,AT&T,80,4
12,T-Mobile,75,3


```SQL
SELECT partner_name, retention_days, bundle_id
FROM(
SELECT
    p.partner_name,
    f.retention_days,
    f.bundle_id,
    RANK() OVER(PARTITION BY p.partner_name ORDER BY f.retention_days DESC) AS rnk
FROM fct_bundle_subscriptions f
JOIN dim_telecom_partners p ON f.partner_id = p.partner_id
WHERE f.conversion_date BETWEEN '2024-10-08' AND '2024-10-14') AS ranked_data
WHERE rnk = 1
```

# Pregunta 3

### "Para los suscriptores que se convirtieron en noviembre de 2024, ¿cuál es el puntaje de compromiso (engagement score) promedio para cada paquete dentro de cada socio de telecomunicaciones?

### ¿Cómo se compara el puntaje promedio de cada paquete con el puntaje de compromiso más alto de toda la historia registrado por su respectivo socio, expresado como un porcentaje de ese máximo?

In [20]:
df_merge['historic_max'] = df_merge.groupby('partner_id')['engagement_score'].transform('max')

df_nov = df_merge[df_merge['conversion_date'].between('2024-11-01','2024-11-30')]

resumen = df_nov.groupby(['partner_name','bundle_id','historic_max'])['engagement_score'].mean().reset_index()

resumen['pct_of_max'] = (resumen['engagement_score'] / resumen['historic_max'] * 100).round(2)

resumen

,partner_name,bundle_id,historic_max,engagement_score,pct_of_max
0,AT&T,3,9.0,8.0,88.89
1,T-Mobile,1,8.6,7.5,87.21
2,T-Mobile,2,8.6,7.2,83.72
3,T-Mobile,4,8.6,4.9,56.98
4,Verizon,2,8.7,8.3,95.40


```SQL
WITH Nov_Stats AS(
    SELECT
        partner_id,
        bundle_id,
        AVG(engagement_score) AS avg_nov_score
    FROM fct_bundle_subscriptions
    WHERE conversion_date BETWEEN '2024-11-01' AND '2024-11-30'
    GROUP BY partner_id, bundle_id
),
Partner_Max AS(
    SELECT
        partner_id,
        MAX(engagement_score) OVER (PARTITION BY partner_id) AS historic_max
    FROM fct_bundle_subscriptions
)
SELECT DISTINCT
    n.partner_id,
    n.bundle_id,
    n.avg_nov_score,
    p.historic_max,
    ROUND((n.avg_nov_score / p.historic_max) * 100, 2) AS pct_of_max
FROM Nov_Stats n
JOIN Partner_Max p ON n.partner_id = p.partner_id;
```